In [ ]:
# !pip install -qqq torch
!pip install -qqq -U transformers==4.30.0
!pip install -qqq einops==0.6.1
!pip install fastapi uvicorn
!pip install pyngrok
!pip install fastapi-cors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from pyngrok import ngrok
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import nest_asyncio

In [ ]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)

cuda


In [ ]:

Model = 'SSTalha/Fashion_PHI_1-5'
model = AutoModelForCausalLM.from_pretrained(Model, torch_dtype="auto", trust_remote_code=True).to(torch_device)
tokenizer = AutoTokenizer.from_pretrained(Model, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

configuration_mixformer_sequential.py:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/typeof/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_mixformer_sequential.py:   0%|          | 0.00/32.3k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [ ]:
def generate_response(user_input):
    # Tokenize the user input
    inputs = tokenizer(user_input, return_tensors="pt", return_attention_mask=False)
    inputs = {key: value.to("cuda") for key, value in inputs.items()}

    outputs = model.generate(**inputs, max_length=120)
    # Decode the generated outputs to get the text
    generated_text = tokenizer.batch_decode(outputs)[0]

    split_text = generated_text.split("###Assistant:")
    if len(split_text) > 1:
        assistant_response = split_text[1].strip()
    else:
        assistant_response = "No valid response found."

    return assistant_response


In [ ]:
app = FastAPI()

app.add_middleware(
CORSMiddleware,
allow_origins=['*'],
allow_credentials=True,
allow_methods=['*'],
allow_headers=['*'],
)

@app.post('/')
async def handle_generate_text(request: Request):
  try:
    # Extract input from request JSON data
    request_data = await request.json()
    user_input = request_data.get('input', '')
    generated_text = generate_response(user_input)
    # Return generated text as response
    return {'generated_text': generated_text}
  except Exception as e:
    return {'error': str(e)}


In [ ]:
auth_token = "2eJXiQZ4fAx562TKuTGBtKi5kG0_7QXVJejkt9Mt9SSdwYcN"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(5000)

print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=5000)

INFO:     Started server process [175]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


Public URL: https://7245-34-143-133-211.ngrok-free.app


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
INFO:     39.45.63.129:0 - "POST / HTTP/1.1" 200 OK
